## Week 7 Homework

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, optimizers, losses

from statistics import median, stdev, mean

In [63]:
## Creating the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Model Architecture
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                11214912  
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
_________________________________________________________________


In [64]:
optmizer = optimizers.SGD(lr=0.002, momentum=0.8)
model.compile(optimizer=optmizer,
              loss=losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [65]:
## Question 3

train_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
     "/home/antihaddock/Repos/mlbookcamp-code/mlbookcamp-code/data/train/",
    seed=1,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary',
    classes=['dino', 'dragon'],
    shuffle=True
)

Found 1594 images belonging to 2 classes.


In [66]:
model.fit(
    train_generator,
    epochs=10
)

Epoch 1/10
80/80 [==============================] - 5s 63ms/step - loss: 0.6294 - accuracy: 0.6344
Epoch 2/10
80/80 [==============================] - 5s 60ms/step - loss: 0.4393 - accuracy: 0.7932
Epoch 3/10
80/80 [==============================] - 5s 60ms/step - loss: 0.3629 - accuracy: 0.8580
Epoch 4/10
80/80 [==============================] - 5s 60ms/step - loss: 0.2891 - accuracy: 0.9011
Epoch 5/10
80/80 [==============================] - 5s 60ms/step - loss: 0.2632 - accuracy: 0.9032
Epoch 6/10
80/80 [==============================] - 5s 59ms/step - loss: 0.2319 - accuracy: 0.9236
Epoch 7/10
80/80 [==============================] - 5s 59ms/step - loss: 0.1964 - accuracy: 0.9355
Epoch 8/10
80/80 [==============================] - 5s 60ms/step - loss: 0.1595 - accuracy: 0.9552
Epoch 9/10
80/80 [==============================] - 5s 60ms/step - loss: 0.1364 - accuracy: 0.9643
Epoch 10/10
80/80 [==============================] - 5s 59ms/step - loss: 0.1091 - accuracy: 0.9796


In [75]:
tf.keras.Model.save(model, 'Homework_model.hdf5')

2022-11-21 20:47:19.819382: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 44859392 exceeds 10% of free system memory.
2022-11-21 20:47:19.911820: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 44859392 exceeds 10% of free system memory.


In [67]:
train_accuracy = model.history.history['accuracy']
median(train_accuracy)

0.9131116569042206

In [68]:
## Question 4
train_loss = model.history.history['loss']
stdev(train_loss)

0.14115357241456608

In [80]:
## Question 5 Prep - training model with new augmented data

# Create augmentor as per the homework question
aug_train_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

aug_train_generator = aug_train_gen.flow_from_directory(
     "/home/antihaddock/Repos/mlbookcamp-code/mlbookcamp-code/data/train/",
    seed=1,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary',
    classes=['dino', 'dragon'],
    shuffle=True
)

# Load in pretrained model to train further
new_model = tf.keras.models.load_model('Homework_model.hdf5')


# # Train for another 10 epoc
# model.fit(
#     train_generator,
#     epochs=10
# )

new_model.fit(
    aug_train_generator,
    epochs=10
)

Found 1594 images belonging to 2 classes.
Epoch 1/10
80/80 [==============================] - 10s 118ms/step - loss: 2765.6101 - accuracy: 0.5157
Epoch 2/10
80/80 [==============================] - 9s 116ms/step - loss: 0.7859 - accuracy: 0.5194
Epoch 3/10
80/80 [==============================] - 9s 116ms/step - loss: 0.6925 - accuracy: 0.5188
Epoch 4/10
80/80 [==============================] - 9s 117ms/step - loss: 0.6925 - accuracy: 0.5188
Epoch 5/10
80/80 [==============================] - 9s 116ms/step - loss: 0.6922 - accuracy: 0.5207
Epoch 6/10
80/80 [==============================] - 9s 117ms/step - loss: 0.6925 - accuracy: 0.5188
Epoch 7/10
80/80 [==============================] - 9s 116ms/step - loss: 0.6935 - accuracy: 0.5182
Epoch 8/10
80/80 [==============================] - 9s 114ms/step - loss: 0.6925 - accuracy: 0.5188
Epoch 9/10
80/80 [==============================] - 9s 113ms/step - loss: 0.6925 - accuracy: 0.5188
Epoch 10/10
80/80 [==============================] - 9

In [90]:
## Question 5 & 6

test_ds =ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_generator = aug_train_gen.flow_from_directory(
     "/home/antihaddock/Repos/mlbookcamp-code/mlbookcamp-code/data/test/",
    seed=1,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary',
    classes=['dino', 'dragon'],
    shuffle=True
)


new_model.evaluate(test_generator)

Found 394 images belonging to 2 classes.
20/20 [==============================] - 3s 129ms/step - loss: 0.6940 - accuracy: 0.4975


[0.6939924359321594, 0.4974619150161743]